In [ ]:
%cd /home/adityasidharta/git/kaggle_salt
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.append('/home/adityasidharta/git/fastkeras')

In [ ]:
from keras.models import load_model

In [ ]:
from python.preprocess import *
from python.losses import *
from python.models import *
from python.metrics import *
from python.config import *
from python.utils import *
from python.predict import *
from python.callbacks import *
from python.threshold import *

In [ ]:
imageloader = ImageLoader(train_path, test_path, depth_file)

In [ ]:
X_train_img = imageloader.get_X_train_img()
X_train_feat = imageloader.get_X_train_feat()
Y_train = imageloader.get_Y_train()
label_train = imageloader.get_label_train()

X_test_img = imageloader.get_X_test_img()
X_test_feat = imageloader.get_X_test_feat()
label_test = imageloader.get_label_test()

In [ ]:
print(X_train_img.shape)
print(X_train_feat.shape)
print(Y_train.shape)
print(X_test_img.shape)
print(X_test_feat.shape)
print(len(label_train))
print(len(label_test))

In [ ]:
X_train = {'img': X_train_img, 'feat': X_train_feat}
X_test = {'img':X_test_img, 'feat':X_test_feat}

In [ ]:
callbacks = get_callbacks('u_net_depthd5')

In [ ]:
u_net_depth = U_NET_DEPTH()
u_net_depth.summary()

In [ ]:
results = u_net_depth.fit(X_train, Y_train, batch_size=32, epochs=500, callbacks=callbacks)

In [ ]:
final_model = load_model('/home/adityasidharta/git/kaggle_salt/model/u_net_depthd5.h5', custom_objects={'dice_coef_loss':dice_coef_loss,
                                                                                               'mean_iou':mean_iou, 'AdamW' : adamw})

In [ ]:
Y_train_pred = final_model.predict(X_train, verbose = 1)

In [ ]:
Y_pred = final_model.predict(X_test, verbose = 1)

In [ ]:
Y_pred = resize_img(Y_pred, ori_n_h, ori_n_v, ori_n_c)

In [ ]:
get_threshold(0.25, 0.75, Y_train, Y_train_pred, 25)

In [ ]:
dict_pred = generate_dict_pred(Y_pred, label_test, 0.5)

In [ ]:
generate_submission(dict_pred, './submissions/u_net_depthd5.csv')